# Cryptopals Challenge Set 2

## 9. Implement PKCS#7 padding

https://cryptopals.com/sets/2/challenges/9

In [1]:
def pkcs7_pad(b: bytes, blocksize: int = 16) -> bytes:
    if blocksize == 16:
        pad_len = blocksize - (len(b) & 15)
    else:
        pad_len = blocksize - (len(b) % blocksize)
    return b + bytes([pad_len]) * pad_len

In [2]:
block = b"YELLOW SUBMARINE"
pkcs7_pad(block,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## 10. Implement CBC mode

https://cryptopals.com/sets/2/challenges/10

https://en.wikipedia.org/wiki/Block_cipher_mode_of_operation#Cipher_block_chaining_(CBC)

In [3]:
from cryptopals.utils import bytes_xor, bytes_to_chuncks
from Cryptodome.Cipher import AES

def aes_cbc_decrypt(cipher: bytes, key: bytes) -> bytes:
    aes = AES.new(key, AES.MODE_ECB)
    bsize = len(key)
    blocks = bytes_to_chuncks(cipher,bsize)
    IV = bsize*b"\x00"    
    plaintext = b""
    for i in range(len(blocks)):
        # decrypt block with AES ECB mode
        plainblock = aes.decrypt(blocks[i])
        # XOR with IV or previous cipher block
        plainblock = bytes_xor(plainblock,IV) if i==0 else bytes_xor(plainblock,blocks[i-1])
        plaintext += plainblock
    return plaintext

In [4]:
from base64 import b64decode

with open("input/10.txt") as f:
    cipher10 = b64decode(f.read())

key = b"YELLOW SUBMARINE"
plaintext10 = aes_cbc_decrypt(cipher10,key)

print(plaintext10.decode())

with open('input/plaintext10.txt', 'w') as f:
    f.write(plaintext10.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## 11. An ECB/CBC detection oracle

https://cryptopals.com/sets/2/challenges/11

> Write a function to generate a random AES key; that's just 16 random bytes. `os.urandom()` return a string of size random bytes suitable for cryptographic use.

https://docs.python.org/3.8/library/os.html#os.urandom

In [5]:
import os

def generate_aes_key(keylen=16):
    return os.urandom(keylen)

generate_aes_key()

b"n\xbfaI\x07\x0e\xf9\x8d\x12\xfc\x16\xab\xc3\x10\x9c'"

> Write a function that encrypts data under an unknown key, e.g. a function that generates a random key and encrypts under it. Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext. Have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). 

> Detect the block cipher mode the function is using each time. 

In [6]:
import os
import random
from Cryptodome.Cipher import AES

def aes_encryption_oracle(plaintext):
    # generate a 16-bytes random key
    keysize = 16
    key = os.urandom(keysize)
    plainb = bytes(plaintext.encode()) # plaintext in bytes
    # prepend and append bytes
    plainb = os.urandom(random.randint(5,10))+plainb+os.urandom(random.randint(5,10)) 
    # pad the plaintext to a multiple of keysize
    if len(plainb)%keysize:
        plainb = pkcs7_pad(plainb,len(plainb)+keysize-len(plainb)%keysize)
    cipher = b""
    mode = random.randint(0,1)
    if mode==1: # encrypt under ECB
        aes_ecb = AES.new(key, AES.MODE_ECB)
        cipher = aes_ecb.encrypt(plainb)
    else: # encrypt under CBC
        IV = os.urandom(keysize)
        aes_cbc = AES.new(key, AES.MODE_CBC, IV) 
        cipher = aes_cbc.encrypt(plainb)
    return mode, cipher

In [7]:
from cryptopals.utils import detect_aes_ecb_mode

# This is the poem provided as ciphertext at challenge 10. No problem in guessing the cipher mode with it!
with open("input/plaintext10.txt") as f:
    plaintext10 = f.read()

for _ in range(10):
    mode, cipher = aes_encryption_oracle(plaintext10) 
    pred = detect_aes_ecb_mode(cipher)
    print(mode,pred)

1 True
0 False
0 False
1 True
0 False
1 True
1 True
1 True
1 True
1 True


## 12. Byte-at-a-time ECB decryption (Simple)

https://cryptopals.com/sets/2/challenges/12

In [8]:
from cryptopals.utils import aes_ecb_decrypt, aes_ecb_encrypt
import os
import random
from base64 import b64decode

class oracle12():
    def __init__(self):
        # key
        self.key = os.urandom(16)
        # target
        self.unknown = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
        self.unknown_bin = b64decode(self.unknown)
        self.unknown_str = self.unknown_bin.decode()
    
    def encrypt(self,string=""):
        '''Encrypt unknown string appending the injection string'''
        return aes_ecb_encrypt(string.encode()+self.unknown_bin,self.key)
    
    def decrypt(self,string):
        return aes_ecb_decrypt(string,self.key).decode()

### Discover the block size of the cipher

In [9]:
def guess_block_size(oracle):
    my_string = ""
    cipher0 = oracle.encrypt(my_string)
    cipher = cipher0
    while len(cipher0)==len(cipher):
        my_string += "A"
        cipher = oracle.encrypt(my_string)
    return len(cipher)-len(cipher0)

In [10]:
oracle = oracle12()
bsize = guess_block_size(oracle)
print("Block size =",bsize)

Block size = 16


### Detect that the function is using ECB

> Feed the oracle function with a repeating string long enough to ensure that repetitions would be visible regardless of the content of the unknown string.

In [11]:
from cryptopals import detect_aes_ecb_mode

my_string = 2*bsize*"A"
my_string_encrypted = oracle.encrypt(my_string)
print("Is the oracle using ECB?", detect_aes_ecb_mode(my_string_encrypted))

Is the oracle using ECB? True


### Attack the oracle with injection string of varying lenght

In [12]:
def find_plain_text_lenght(oracle):
    bsize = guess_block_size(oracle)
    i = 0
    cipher_0 = oracle.encrypt("")
    cipher_i = cipher_0
    for i in range(bsize):
        cipher_i = oracle.encrypt(i*"A")
        if len(cipher_0)<len(cipher_i):
            return len(cipher_0)-i+1

def byte_at_a_time_ecb_decrypt(oracle):
    injsize = len(oracle.encrypt(""))
    plaintext = ""
    for k in range(injsize):
        my_string = (injsize-1-len(plaintext))*"A"
        cipher_0 = oracle.encrypt(my_string)
        for i in range(256):
            cipher_i = oracle.encrypt(my_string+plaintext+chr(i))
            if cipher_0[:injsize] == cipher_i[:injsize]:
                plaintext += chr(i)
                break
    return plaintext[:find_plain_text_lenght(oracle)]

In [13]:
oracle = oracle12()
plaintext12 = byte_at_a_time_ecb_decrypt(oracle)
print(plaintext12)
print(oracle.unknown_str)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



## 13. ECB cut-and-paste

https://cryptopals.com/sets/2/challenges/13

In [14]:
from cryptopals import aes_ecb_decrypt, aes_ecb_encrypt, pkcs7_strip
import os

class profile_functions_13:
    def __init__(self):
        self.key = os.urandom(16)
        
    def parse(self,string):
        '''k=v parsing routine'''
        return { i.split("=")[0]: i.split("=")[1] for i in string.split("&") }
    
    def profile_for(self,email):
        # Your "profile_for" function should not allow encoding metacharacters (& and =). 
        if b"&" in email.encode() or b"=" in email.encode():
            #raise ValueError("Invalid email address")
            print("Invalid email address")
            return b""
        return b"email=" + email.encode() + b"&uid=10&role=user"
    
    def encrypt(self,email):
        profile = self.profile_for(email)
        if profile != "":
            return aes_ecb_encrypt(profile,self.key)

    def decrypt(self,encprof):
        return pkcs7_strip(aes_ecb_decrypt(encprof,self.key)).decode()

In [15]:
profile = profile_functions_13()
print(profile.parse("foo=bar&baz=qux&zap=zazzle"))
print(profile.profile_for("foo@bar.com"))

{'foo': 'bar', 'baz': 'qux', 'zap': 'zazzle'}
b'email=foo@bar.com&uid=10&role=user'


In [16]:
encprof = profile.encrypt("foo@bar.com")
print(profile.decrypt(encprof))

email=foo@bar.com&uid=10&role=user


### Attack

* Forging a fake email to get the `user` role at the beginning of a block. 
* Since blocks are encoded separately I could then replace that (encoded) `user` block only with the (encoded) `admin` role.


In [17]:
from cryptopals import bytes_to_chuncks, pkcs7_pad

attack_target = b'user'
attack_role   = b'admin'

# guessing the block size
bsize = guess_block_size(profile)

# forging an attack email matching the block size
attack_email  = "foo@bar.com"
while True:
    prof = profile.profile_for(attack_email)
    if bytes_to_chuncks(prof,bsize)[-1]==attack_target:
        break
    attack_email = "f"+attack_email

# encrypted attack email
cipher_user = profile.encrypt(attack_email)

# padding attack role to fill one block
attack_block = pkcs7_pad(attack_role,bsize)

# devoting two blocks to forged email, so that attack role (admin) will be at tbe beginning of third block
prepend = "email="
attack_email_2 = (bsize-len(prepend))*b"f"+(bsize-len(attack_email))*b"f"+attack_email.encode()
prof_attack = profile.profile_for( (attack_email_2+attack_block).decode() )

# getting the cipher for the forged email address
encprof_attackrole = profile.encrypt( (attack_email_2+attack_block).decode() )
bytes_to_chuncks(encprof_attackrole,bsize)[2]

# merging blocks from forged 'user' profile with block from admin attack
encprof_attack = b"".join(bytes_to_chuncks(cipher_user,bsize)[:2]+[bytes_to_chuncks(encprof_attackrole,bsize)[2]] )

# attack result
profile.decrypt(encprof_attack)

'email=fffoo@bar.com&uid=10&role=admin'

## 14. Byte-at-a-time ECB decryption (Harder)

https://cryptopals.com/sets/2/challenges/14

> The oracle should now "generate a random count of random bytes and prepend this string to every plaintext". Shall the prefix string be fixed or change at any call of the oracle? Assuming it's a salting procedure, I would say it should always be the same...

In [18]:
from cryptopals import aes_ecb_decrypt, aes_ecb_encrypt
import os
import random
from base64 import b64decode

class oracle14:
    def __init__(self):
        # key
        self.key = os.urandom(16)
        # random prefix
        self.prefix = os.urandom(random.randint(1,15))
        # target
        self.unknown = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
        self.unknown_bin = b64decode(self.unknown)
        self.unknown_str = self.unknown_bin.decode()
        
    def encrypt(self, plaintext="", randomSalt=False):
        plainb = self.prefix + plaintext.encode() + self.unknown_bin
        return aes_ecb_encrypt(plainb,self.key)
        
    def decrypt(self,string):
        return aes_ecb_decrypt(string,self.key).decode()

In [19]:
oracle = oracle14()
bsize = guess_block_size(oracle)
print("Block size:",bsize)

Block size: 16


### Finding the prefix lenght

In [20]:
def find_prefix_lenght(oracle):
    bsize = guess_block_size(oracle)
    cipher0 = oracle.encrypt("")
    cipher1 = oracle.encrypt("A")
    blocks0 = bytes_to_chuncks(cipher0,bsize)
    blocks1 = bytes_to_chuncks(cipher1,bsize)
    common = sum([ 1 if b0==b1 else 0 for b0,b1 in zip(blocks0,blocks1) ])
    i = 0
    while True:
        cipher0 = oracle.encrypt(i*"A")
        cipher1 = oracle.encrypt((i+1)*"A")
        blocks0 = bytes_to_chuncks(cipher0,bsize)
        blocks1 = bytes_to_chuncks(cipher1,bsize)
        sameBlock = 0 
        for b0,b1 in zip(blocks0,blocks1):
            if b0==b1:
                sameBlock += 1
        if sameBlock>common:
            return sameBlock*bsize-i
        i += 1
        
prefixlen = find_prefix_lenght(oracle)
assert(prefixlen==len(oracle.prefix))
print("Prefix lenght:",prefixlen)

Prefix lenght: 1


### Attack

In [21]:
def find_plain_text_lenght_prefix(oracle):
    bsize = guess_block_size(oracle)
    prefixlen = find_prefix_lenght(oracle)
    i = 0
    cipher_0 = oracle.encrypt("")
    cipher_i = cipher_0
    for i in range(bsize):
        cipher_i = oracle.encrypt(i*"A")
        if len(cipher_i)>len(cipher_0):
            return len(cipher_0)-i+1-prefixlen

def byte_at_a_time_ecb_decryption_prefix(oracle):
    prefixlen = find_prefix_lenght(oracle)
    injsize = len(oracle.encrypt(""))-prefixlen
    plaintext = ""
    for k in range(injsize):
        my_string = (injsize-1-len(plaintext))*"A"
        cipher_0 = oracle.encrypt(my_string)
        for i in range(256):
            cipher_i = oracle.encrypt(my_string+plaintext+chr(i))
            if cipher_0[:injsize] == cipher_i[:injsize]:
                plaintext += chr(i)
                break
    return plaintext[:find_plain_text_lenght_prefix(oracle)]

In [22]:
oracle = oracle14()
plaintext14 = byte_at_a_time_ecb_decryption_prefix(oracle)
print(plaintext14)
print(oracle.unknown_str)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



## 15. PKCS#7 padding validation

https://cryptopals.com/sets/2/challenges/15

In [23]:
s1 = b"ICE ICE BABY\x04\x04\x04\x04"
s2 = b"ICE ICE BABY\x05\x05\x05\x05"
s3 = b"ICE ICE BABY\x01\x02\x03\x04"

In [24]:
class PaddingError(Exception):
    pass
    
def pkcs7_strip(b: bytes) -> bytes:
    n = b[-1]
    if n==0 or len(b)<n or not b.endswith(bytes([n])*n): # invalid padding
        raise PaddingError
    else:
        return b[:-n]

In [25]:
assert pkcs7_strip(b"ICE ICE BABY\x04\x04\x04\x04") == b'ICE ICE BABY'

try:
    pkcs7_strip(s1)
except PaddingError:
    print("Padding Error!")
    pass
else:
    print("Padding correct!")

Padding correct!


In [26]:
try:
    pkcs7_strip(s2)
except PaddingError:
    print("Padding Error!")
    pass
else:
    print("Padding correct!")

Padding Error!


In [27]:
try:
    pkcs7_strip(s3)
except PaddingError:
    print("Padding Error!")
    pass
else:
    print("Padding correct!")

Padding Error!


## 16. CBC bitflipping attacks

https://cryptopals.com/sets/2/challenges/16

In [28]:
from Cryptodome.Cipher import AES
from cryptopals import pkcs7_pad, pkcs7_strip, bytes_xor
import os

BLOCKSIZE = 16
KEYSIZE = 32

class profile_functions_16:
    def __init__(self):
        self.key = os.urandom(KEYSIZE)
        self.iv = os.urandom(BLOCKSIZE)
        
    def wrap_userdata(self, data: bytes) -> bytes:
        prefix = b"comment1=cooking%20MCs;userdata="
        suffix = b";comment2=%20like%20a%20pound%20of%20bacon"
        data = data.replace(b";",b"%3B").replace(b"=",b"%3D") # The function should quote out the ";" and "=" characters.
        wrapped = prefix + data + suffix
        aes_cbc = AES.new(self.key,AES.MODE_CBC,self.iv)
        return aes_cbc.encrypt(pkcs7_pad(wrapped))
    
    def check_for_admin(self, data: bytes, quiet=False) -> bool:
        aes_cbc = AES.new(self.key,AES.MODE_CBC,self.iv)
        plaintext = pkcs7_strip(aes_cbc.decrypt(data))
        if not quiet:
            print(f"{plaintext=}")
        return b";admin=true;" in plaintext

In [29]:
profile = profile_functions_16()
cipher1 = profile.wrap_userdata(b";admin=true;")
cipher2= profile.wrap_userdata(b"A"*BLOCKSIZE*2)
profile.check_for_admin(cipher1,False)
profile.check_for_admin(cipher2,False)

plaintext=b'comment1=cooking%20MCs;userdata=%3Badmin%3Dtrue%3B;comment2=%20like%20a%20pound%20of%20bacon'
plaintext=b'comment1=cooking%20MCs;userdata=AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA;comment2=%20like%20a%20pound%20of%20bacon'


False

In [30]:
def make_cbc_bitflipping_attack(profile) -> bytes:
    a_block = b"A" * BLOCKSIZE
    cipher = profile.wrap_userdata(2*a_block)
    # right justify injection block with padding
    injection = b";admin=true".rjust(BLOCKSIZE, b"A")
    flipper = bytes_xor(a_block,injection)
    # flipped block will be 4th block in plain text, it's then left justified to the lenght of the ciphertext
    padded = flipper.rjust(3*BLOCKSIZE, b"\x00").ljust(len(cipher), b"\x00")
    # xor with original encrypter wrapped user data
    cipher_new = bytes_xor(cipher,padded)
    return cipher_new

In [31]:
profile = profile_functions_16()
attack_data = make_cbc_bitflipping_attack(profile)
profile.check_for_admin(attack_data,False)

plaintext=b'comment1=cooking%20MCs;userdata=\x1f\\\xb2\xfbZ\xcft\xcc\x0e\x13M\xb5\x82@\x92\xd7AAAAA;admin=true;comment2=%20like%20a%20pound%20of%20bacon'


True